# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GNNTrainer** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from gnn import GNNTrainer
from movie_lens_loader import MovieLensLoader
from llm import PromptBertClassifier, VanillaBertClassifier, AddingEmbeddingsBertClassifierBase

from transformers import AutoConfig

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [2]:
MODEL_NAME = "google/bert_uncased_L-2_H-128_A-2"
MODEL_HIDDEN_SIZE = AutoConfig.from_pretrained(MODEL_NAME).hidden_size
KGE_DIMENSIONS = [4] # Output Dimension of the GNN Encoder.
EPOCHS = 20
BATCH_SIZE = 256

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-latest-small.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [3]:

movie_lens_loader = MovieLensLoader(kge_dimensions = KGE_DIMENSIONS)

In [4]:
movie_lens_loader.llm_df.head()

,mappedUserId,mappedMovieId,title,genres,prompt,split,user_embedding_4,movie_embedding_4,user_embedding_128,movie_embedding_128
0,0,0,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy...","user: 0, title: Toy Story (1995), genres: ['Ad...",train,"[-0.03827861696481705, -2.2619593143463135, 2....","[-0.550093412399292, -0.8870471119880676, 0.99...","[-0.20877254009246826, 0.7261011004447937, -0....","[-0.23873792588710785, 0.15261110663414001, -0..."
1,0,2,Grumpier Old Men (1995),"['Comedy', 'Romance']","user: 0, title: Grumpier Old Men (1995), genre...",rest,"[0.33092743158340454, -2.609788417816162, 2.80...","[-0.14393314719200134, -1.0467438697814941, -0...","[-0.3288712799549103, 0.7533886432647705, -0.2...","[-0.601038932800293, -0.014170907437801361, -0..."
2,0,5,Heat (1995),"['Action', 'Crime', 'Thriller']","user: 0, title: Heat (1995), genres: ['Action'...",val,"[0.26843586564064026, -2.6071953773498535, 2.8...","[-0.5415042638778687, -0.8318737745285034, 0.6...","[-0.26235586404800415, 0.8743588328361511, -0....","[-0.00794781744480133, -0.03061569482088089, 0..."
3,0,43,Seven (a.k.a. Se7en) (1995),"['Mystery', 'Thriller']","user: 0, title: Seven (a.k.a. Se7en) (1995), g...",train,"[0.18218860030174255, -2.3380637168884277, 2.7...","[-0.9858976602554321, -0.801932692527771, 0.80...","[-0.23487024009227753, 0.6445444226264954, -0....","[0.23825839161872864, 0.006370842456817627, -0..."
4,0,46,"Usual Suspects, The (1995)","['Crime', 'Mystery', 'Thriller']","user: 0, title: Usual Suspects, The (1995), ge...",test,"[0.09569544345140457, -1.986099123954773, 3.03...","[-0.865667998790741, -0.9774402976036072, 0.52...","[-0.1536864936351776, 0.8239970803260803, -0.3...","[0.16465282440185547, 0.09803374111652374, -0...."


Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [5]:
gnn_trainers =    [GNNTrainer(movie_lens_loader.data, kge_dimension = kge_dimension) for kge_dimension in KGE_DIMENSIONS]
gnn_trainer_large = GNNTrainer(movie_lens_loader.data, hidden_channels=MODEL_HIDDEN_SIZE, kge_dimension=MODEL_HIDDEN_SIZE)

loading pretrained model
Device: 'cuda'
loading pretrained model
Device: 'cuda'


We then train and validate the model on the link prediction task.

If the model is already trained, we can skip this part.
Training the models can take up to 5 minutes.

In [6]:
for gnn_trainer in gnn_trainers:
    print(gnn_trainer.kge_dimension)
    #gnn_trainer.train_model(movie_lens_loader.gnn_train_data, EPOCHS)
    #gnn_trainer.validate_model(movie_lens_loader.gnn_test_data)
#print("large_gnn")
#gnn_trainer_large.train_model(movie_lens_loader.gnn_train_data, EPOCHS)
#gnn_trainer_large.validate_model(movie_lens_loader.gnn_test_data)


4


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

In [7]:
[gnn_trainer.get_embeddings(movie_lens_loader) for gnn_trainer in gnn_trainers]
gnn_trainer_large.get_embeddings(movie_lens_loader)
movie_lens_loader.llm_df.head()

,mappedUserId,mappedMovieId,title,genres,prompt,split,user_embedding_4,movie_embedding_4,user_embedding_128,movie_embedding_128
0,0,0,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy...","user: 0, title: Toy Story (1995), genres: ['Ad...",train,"[-0.03827861696481705, -2.2619593143463135, 2....","[-0.550093412399292, -0.8870471119880676, 0.99...","[-0.20877254009246826, 0.7261011004447937, -0....","[-0.23873792588710785, 0.15261110663414001, -0..."
1,0,2,Grumpier Old Men (1995),"['Comedy', 'Romance']","user: 0, title: Grumpier Old Men (1995), genre...",rest,"[0.33092743158340454, -2.609788417816162, 2.80...","[-0.14393314719200134, -1.0467438697814941, -0...","[-0.3288712799549103, 0.7533886432647705, -0.2...","[-0.601038932800293, -0.014170907437801361, -0..."
2,0,5,Heat (1995),"['Action', 'Crime', 'Thriller']","user: 0, title: Heat (1995), genres: ['Action'...",val,"[0.26843586564064026, -2.6071953773498535, 2.8...","[-0.5415042638778687, -0.8318737745285034, 0.6...","[-0.26235586404800415, 0.8743588328361511, -0....","[-0.00794781744480133, -0.03061569482088089, 0..."
3,0,43,Seven (a.k.a. Se7en) (1995),"['Mystery', 'Thriller']","user: 0, title: Seven (a.k.a. Se7en) (1995), g...",train,"[0.18218860030174255, -2.3380637168884277, 2.7...","[-0.9858976602554321, -0.801932692527771, 0.80...","[-0.23487024009227753, 0.6445444226264954, -0....","[0.23825839161872864, 0.006370842456817627, -0..."
4,0,46,"Usual Suspects, The (1995)","['Crime', 'Mystery', 'Thriller']","user: 0, title: Usual Suspects, The (1995), ge...",test,"[0.09569544345140457, -1.986099123954773, 3.03...","[-0.865667998790741, -0.9774402976036072, 0.52...","[-0.1536864936351776, 0.8239970803260803, -0.3...","[0.16465282440185547, 0.09803374111652374, -0...."


Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [8]:
vanilla_bert_classifier = VanillaBertClassifier(movie_lens_loader.llm_df, batch_size=BATCH_SIZE, model_name=MODEL_NAME)

Next we generate a vanilla llm dataset and tokenize it for training.

In [9]:
dataset_vanilla = movie_lens_loader.generate_vanilla_dataset(vanilla_bert_classifier.tokenize_function)

Next we train the model on the produced dataset. This can be skipped, if already trained ones.

In [10]:
#vanilla_bert_classifier.train_model_on_data(dataset_vanilla, epochs=EPOCHS)

Next we initialize the prompt encoder only classifier. This classifier uses the vanilla prompt and the KGEs for its link prediction.

In [11]:
prompt_bert_classifiers = [PromptBertClassifier(movie_lens_loader, gnn_trainer.get_embedding, kge_dimension=embedding_dimension, batch_size=BATCH_SIZE, model_name=MODEL_NAME) for embedding_dimension, gnn_trainer in zip(KGE_DIMENSIONS, gnn_trainers)]

We also generate a prompt dataset, this time the prompts also include the KGEs.

In [12]:
datasets_prompt = [movie_lens_loader.generate_prompt_embedding_dataset(prompt_bert_classifier.tokenize_function, kge_dimension = prompt_bert_classifier.kge_dimension) for prompt_bert_classifier in prompt_bert_classifiers]

We also train the model. This can be skipped if already done ones.

In [13]:
#[prompt_bert_classifier.train_model_on_data(dataset_prompt, epochs = EPOCHS) for prompt_bert_classifier, dataset_prompt in zip(prompt_bert_classifiers, datasets_prompt)]

In [14]:
adding_embedding_bert_only_classifier = AddingEmbeddingsBertClassifierBase(movie_lens_loader, gnn_trainer_large.get_embedding, kge_dimension=MODEL_HIDDEN_SIZE, batch_size=BATCH_SIZE, model_name=MODEL_NAME)
dataset_adding_embedding = movie_lens_loader.generate_adding_embedding_dataset(adding_embedding_bert_only_classifier.tokenizer.sep_token, adding_embedding_bert_only_classifier.tokenizer.pad_token, adding_embedding_bert_only_classifier.tokenize_function, kge_dimension = MODEL_HIDDEN_SIZE)


Some weights of InsertEmbeddingBertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
adding_embedding_bert_only_classifier.train_model_on_data(dataset_adding_embedding, epochs = EPOCHS)

  0%|          | 0/4420 [00:00<?, ?it/s]

c:\Users\MARS\.conda\envs\hauptprojekt\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.7771, 'grad_norm': 3.4389734268188477, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.05}
{'loss': 0.7699, 'grad_norm': 3.764988422393799, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.09}
{'loss': 0.7705, 'grad_norm': 3.20927357673645, 'learning_rate': 3e-06, 'epoch': 0.14}
{'loss': 0.7597, 'grad_norm': 3.2560675144195557, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.18}
{'loss': 0.7518, 'grad_norm': 2.8689229488372803, 'learning_rate': 5e-06, 'epoch': 0.23}
{'loss': 0.7356, 'grad_norm': 3.503141403198242, 'learning_rate': 6e-06, 'epoch': 0.27}
{'loss': 0.7217, 'grad_norm': 2.7570619583129883, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.32}
{'loss': 0.7035, 'grad_norm': 1.7646822929382324, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.36}
{'loss': 0.6869, 'grad_norm': 0.8705242276191711, 'learning_rate': 9e-06, 'epoch': 0.41}
{'loss': 0.6647, 'grad_norm': 1.5016316175460815, 'learning_rate': 1e-05, 'epoch': 0.45}
{'loss': 0.6443, 'grad_norm': 0.

  0%|          | 0/67 [00:00<?, ?it/s]

{'eval_loss': 0.587656557559967, 'eval_accuracy': 0.6646248979115622, 'eval_runtime': 157.3055, 'eval_samples_per_second': 108.973, 'eval_steps_per_second': 0.426, 'epoch': 1.0}
{'loss': 0.6019, 'grad_norm': 0.8464211225509644, 'learning_rate': 2.3000000000000003e-05, 'epoch': 1.04}
{'loss': 0.5844, 'grad_norm': 0.8668562173843384, 'learning_rate': 2.4e-05, 'epoch': 1.09}
{'loss': 0.5756, 'grad_norm': 1.1645541191101074, 'learning_rate': 2.5e-05, 'epoch': 1.13}
{'loss': 0.5514, 'grad_norm': 0.6494861245155334, 'learning_rate': 2.6000000000000002e-05, 'epoch': 1.18}
{'loss': 0.555, 'grad_norm': 0.900191068649292, 'learning_rate': 2.7000000000000002e-05, 'epoch': 1.22}
{'loss': 0.5468, 'grad_norm': 1.0752432346343994, 'learning_rate': 2.8000000000000003e-05, 'epoch': 1.27}
{'loss': 0.5371, 'grad_norm': 1.3139674663543701, 'learning_rate': 2.9e-05, 'epoch': 1.31}
{'loss': 0.5096, 'grad_norm': 1.4389201402664185, 'learning_rate': 3e-05, 'epoch': 1.36}
{'loss': 0.5249, 'grad_norm': 1.339485

...

{'eval_loss': 0.2539231479167938, 'eval_accuracy': 0.8931279897328199, 'eval_runtime': 160.3263, 'eval_samples_per_second': 106.919, 'eval_steps_per_second': 0.418, 'epoch': 18.0}
{'loss': 0.2778, 'grad_norm': 1.154860258102417, 'learning_rate': 5.612244897959184e-06, 'epoch': 18.01}
{'loss': 0.2791, 'grad_norm': 1.1577537059783936, 'learning_rate': 5.48469387755102e-06, 'epoch': 18.05}
{'loss': 0.2964, 'grad_norm': 0.9296960830688477, 'learning_rate': 5.357142857142857e-06, 'epoch': 18.1}
{'loss': 0.2751, 'grad_norm': 0.9612064361572266, 'learning_rate': 5.229591836734695e-06, 'epoch': 18.14}
{'loss': 0.2617, 'grad_norm': 1.0373328924179077, 'learning_rate': 5.102040816326531e-06, 'epoch': 18.19}
{'loss': 0.2788, 'grad_norm': 2.3092970848083496, 'learning_rate': 4.9744897959183674e-06, 'epoch': 18.24}
{'loss': 0.2612, 'grad_norm': 1.8288300037384033, 'learning_rate': 4.846938775510204e-06, 'epoch': 18.28}
{'loss': 0.26, 'grad_norm': 0.9837968945503235, 'learning_rate': 4.7193877551020

  0%|          | 0/67 [00:00<?, ?it/s]

{'eval_loss': 0.25330156087875366, 'eval_accuracy': 0.8917279197293198, 'eval_runtime': 159.9405, 'eval_samples_per_second': 107.177, 'eval_steps_per_second': 0.419, 'epoch': 19.0}
{'loss': 0.2612, 'grad_norm': 1.0151054859161377, 'learning_rate': 2.806122448979592e-06, 'epoch': 19.0}
{'loss': 0.2966, 'grad_norm': 1.208803415298462, 'learning_rate': 2.6785714285714285e-06, 'epoch': 19.05}
{'loss': 0.2811, 'grad_norm': 1.0914499759674072, 'learning_rate': 2.5510204081632653e-06, 'epoch': 19.1}
{'loss': 0.2747, 'grad_norm': 0.899793267250061, 'learning_rate': 2.423469387755102e-06, 'epoch': 19.14}
{'loss': 0.2777, 'grad_norm': 0.741868257522583, 'learning_rate': 2.295918367346939e-06, 'epoch': 19.19}
{'loss': 0.2847, 'grad_norm': 0.8857210278511047, 'learning_rate': 2.1683673469387757e-06, 'epoch': 19.23}
{'loss': 0.2958, 'grad_norm': 0.9391604065895081, 'learning_rate': 2.040816326530612e-06, 'epoch': 19.28}
{'loss': 0.2577, 'grad_norm': 0.8202571272850037, 'learning_rate': 1.9132653061

  0%|          | 0/67 [00:00<?, ?it/s]

{'eval_loss': 0.2521660327911377, 'eval_accuracy': 0.8913195659782989, 'eval_runtime': 160.1167, 'eval_samples_per_second': 107.059, 'eval_steps_per_second': 0.418, 'epoch': 20.0}
{'train_runtime': 14229.5051, 'train_samples_per_second': 79.369, 'train_steps_per_second': 0.311, 'train_loss': 0.3319284814515265, 'epoch': 20.0}
